In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT_FOLDER = "NYPL-menus"

MENU = 1
MENU_PAGE = 3
MENU_ITEM = 2
DISH = 0

In [3]:
dataset = []

for filename in sorted(Path(ROOT_FOLDER).iterdir()):
    if filename.name.endswith(".csv"):
        dataset.append(pd.read_csv(filename, na_values=[""]))

# dataset[MENU].head()
# dataset[MENU_PAGE].head()
# dataset[MENU_ITEM].head()
# dataset[DISH].head()

In [28]:
# IC 1: Temporal consistency in Dish
ic1_violations = dataset[DISH][dataset[DISH]["first_appeared"] > dataset[DISH]["last_appeared"]]

print(f"Violations found: {len(ic1_violations)}")
ic1_violations.head(10)

Violations found: 6


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
131193,164029,Clear beef broth,NaN,0,1,1900,0,0.25,0.25
163257,204888,Hot roast beef with gravy,NaN,0,1,1900,0,0.25,0.25
197050,250693,SURI LEBERLI - Shredded Calf's Liver Flambe in...,NaN,0,1,1945,0,NaN,NaN
197053,250699,"SWISS MINCED VEAL, ROESTI",NaN,0,1,1945,0,NaN,NaN
237740,301736,Cafe Glacee,NaN,0,2,1940,0,0.40,0.40
244534,309629,Garlic Butter,NaN,0,1,1947,0,0.40,0.40


In [ ]:
# IC 2: Temporal consistency in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: call_prefix == date_year when both exist
ic2_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() & 
    dataset[MENU]["date"].notna() & 
    dataset[MENU]["call_prefix"].str.isnumeric() &
    dataset[MENU]["date"].str.match(r"[^1]+") & 
    (dataset[MENU]["call_prefix"] != dataset[MENU]["date_prefix"])
]

print(f"Violations found: {len(ic2_violations)}")
ic2_violations[["id", "call_number", "date", "call_prefix", "date_prefix"]].head()

Violations found: 4


,id,call_number,date,call_prefix,date_prefix
525,13112,1900-2328,0190-03-06,1900,0190
14659,32265,1918-0387_wotm,2928-03-26,1918,2928
16917,34727,1912-0667_wotm,0001-01-01,1912,0001
16918,34728,1912-0668_wotm,0001-01-01,1912,0001
